In [ ]:
from getpass import getpass
import os
from dotenv import load_dotenv

# 環境變數
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

In [ ]:
from typing import List
from pydantic import BaseModel

class City(BaseModel):
    # 城市名稱
    name: str
    # 國家
    country: str
    # 人口
    population: int

class CitiesData(BaseModel):
    # 城市列表
    cities: List[City]

In [ ]:
json_schema = CitiesData.schema_json(indent=2)
json_schema

In [ ]:
import json
import random
import pydantic
from pydantic import ValidationError
from typing import Optional, List
from colorama import Fore
from haystack import component

# 定義組件的輸入參數
@component
class OutputValidator:
    def __init__(self, pydantic_model: pydantic.BaseModel):
        self.pydantic_model = pydantic_model  # 儲存 Pydantic 模型
        self.iteration_counter = 0  # 初始化循環計數器

    # 定義組件的輸出
    @component.output_types(valid_replies=List[str], invalid_replies=Optional[List[str]], error_message=Optional[str])
    def run(self, replies: List[str]):
        self.iteration_counter += 1  # 增加循環計數器

        ## 嘗試解析 LLM 的回覆 ##
        # 如果 LLM 的回覆是一個有效的對象，返回 `"valid_replies"`
        try:
            output_dict = json.loads(replies[0])  # 解析回覆為字典
            self.pydantic_model.parse_obj(output_dict)  # 使用 Pydantic 模型進行驗證
            print(
                Fore.GREEN
                + f"OutputValidator at Iteration {self.iteration_counter}: Valid JSON from LLM - No need for looping: {replies[0]}"
            )
            return {"valid_replies": replies}

        # 如果 LLM 的回覆損壞或無效，返回 "invalid_replies" 和 "error_message" 以便 LLM 重試
        except (ValueError, ValidationError) as e:
            print(
                Fore.RED
                + f"OutputValidator at Iteration {self.iteration_counter}: Invalid JSON from LLM - Let's try again.\n"
                f"Output from LLM:\n {replies[0]} \n"
                f"Error from OutputValidator: {e}"
            )
            return {"invalid_replies": replies, "error_message": str(e)}

In [ ]:
output_validator = OutputValidator(pydantic_model=CitiesData)
print(output_validator)

In [ ]:
prompt_template = """
    根據此段落中存在的資訊建立一個 JSON 對象：
    {{passage}}.
    僅使用段落中存在的資訊。
    遵循此 JSON 架構，但僅傳回實際實例，無需任何其他架構定義：
    {{schema}}
    確保自己的答案是字典而不是列表。
    {% if invalid_replies and error_message %}
    您在先前的嘗試中已經建立了以下輸出：
    {{invalid_replies}}
    但是，這不符合上面的格式要求並觸發了此 Python 異常：
    {{error_message}}
    更正輸出並重試。只需返回正確的輸出，無需任何額外的解釋。
    {% endif %}
"""

In [ ]:
from haystack.components.builders import PromptBuilder

prompt_builder = PromptBuilder(template=prompt_template)

In [ ]:
from haystack.components.generators import OpenAIGenerator

# 初始化 OpenAI 生成器
generator = OpenAIGenerator(model="gpt-4-turbo")

In [ ]:
from haystack import Pipeline

pipeline = Pipeline(max_loops_allowed=5)

In [ ]:
# 添加組件到管道
pipeline.add_component(
    instance=prompt_builder, name="prompt_builder"
)
pipeline.add_component(
    instance=generator, name="llm"
)
pipeline.add_component(
    instance=output_validator, name="output_validator"
)

In [ ]:
# 現在，將組件之間進行連接
pipeline.connect(
    "prompt_builder", "llm"
)
pipeline.connect(
    "llm", "output_validator"
)
# 如果一個組件有多個輸出或輸入，請明確指定連接：
pipeline.connect(
    "output_validator.invalid_replies",
    "prompt_builder.invalid_replies"
)
pipeline.connect(
    "output_validator.error_message",
    "prompt_builder.error_message"
)

In [ ]:
pipeline.draw("auto-correct-pipeline.png")
from utils.draw_pipeline import draw_and_display

draw_and_display(pipeline, "ex08_1_pipe.png")

In [ ]:
passage = "柏林是德國的首都。人口3,850,809。法國首都巴黎，居民216.1萬。里斯本是葡萄牙首都和最大城市，人口 504,718 人。"

In [ ]:
result = pipeline.run({
    "prompt_builder": {
        "passage": passage,
        "schema": json_schema
    }
})

In [ ]:
valid_reply = result["output_validator"]["valid_replies"][0]
valid_json = json.loads(valid_reply)
print(valid_json)

更複雜的例子

In [ ]:
introduction_tw = "臺灣（俗字寫作台灣），西方國家亦稱福爾摩沙（葡萄牙語：Formosa），是位於東亞、太平洋西北側的島嶼，地處琉球群島與菲律賓群島之間，西隔臺灣海峽與中國大陸相望，海峽距離約130公里，周圍海域從3點鐘方向以順時鐘排序分別為太平洋（菲律賓海）、巴士海峽、南海、臺灣海峽、東海。面積約3.6萬平方公里，在當前全球各島嶼面積排名中位居第38（或39），島上約七成面積之地形為山地與丘陵，平原主要集中於西部沿海，地形海拔變化大，最高點3952公尺。全島被北迴歸線貫穿，氣候炎熱，夏季偏長，介於熱帶與亞熱帶地帶之間，北迴歸線以北為副熱帶季風氣候、以南為熱帶季風氣候[6][7]，自然景觀與生態系資源豐富多元[8]。臺灣面積約3.6萬平方公里，在當前全球各島嶼面積排名中位居第38（或39），為板塊碰撞隆起形成的大陸島，是東亞島弧之一部分（由菲律賓板塊潛入歐亞板塊形成）。島上約七成面積之地形為山地與丘陵，平原主要集中於西部沿海，地形海拔變化大，最高點3952公尺。全島被北迴歸線貫穿，氣候炎熱，夏季偏長，介於熱帶與亞熱帶地帶之間，北迴歸線以北為副熱帶季風氣候、以南為熱帶季風氣候[9][10]，自然景觀與生態系資源豐富多元[8]。人口約2300萬人，超過七成集中於西部的五大都會區，其中以行政中心臺北為核心的臺北都會區最大，約700萬人。族群構成以漢族、原住民族為主：原住民族由多個屬於南島民族的部族組成，漢族則依民系及移民年代的不同而分為閩南（河洛）、客家與外省族群，其中閩南裔為臺灣最大族群。約三萬年前冰河時期開始有人類遷移至台灣活動，自古為原住民族世居之地[11]，原住民族在17世紀中葉以前一直居於主體民族地位；隨著漢族不斷從中國大陸移入與墾殖，漢族遂取代原住民族成為臺灣的最大民族。自有信史記錄以來，臺灣歷史上曾經歷多個原住民聯盟和政權、荷西時期、明鄭時期、清治時期、日治時期等多次政權遞嬗，最近一次為1945年進入戰後時期由中華民國統治。1949年中華民國政府播遷臺灣造成兩岸分治的局面後，臺灣成為中華民國有效統治領土的主要部分[d]。隨著1987年戒嚴時代結束，臺灣逐漸淡化過往戒嚴時代形塑的中國史觀，政治上走向自由化與民主化，以中國國民黨及民主進步黨兩黨為首的政黨政治、統獨議題、以及公民社會的形成，加之以東南亞新住民的定居，產生出多元文化主義[12][13]，使得臺灣文化呈現多元並立的面貌。而由此原因再加上一中原則，使得現今「臺灣」成為中華民國的通稱。歷經1860年臺灣開港以來至日治時期所打下的現代化基礎[14]，以及中華民國政府遷臺後運用美援所進行的一系列的經濟建設，加上國際上冷戰對峙的格局，臺灣自1960年代起在經濟與社會發展上突飛猛進，締造「臺灣奇蹟」，名列亞洲四小龍之一；之後在1990年代躋身已開發國家之列，目前無論人均所得或人類發展指數均具世界先進國家水準[15]。臺灣擁有蓬勃的製造業及尖端科技，在半導體、資訊科技、通訊、電子精密製造等領域執牛耳。貿易方面主要透過高科技產業賺取外匯，經濟發展上以高科技產業與服務業為中心，亦朝向文化產業及觀光業發展[16]。台灣以移民為主的人文結構，亦帶來多元的政治觀點。自大航海時代以來，台灣文化就在明鄭、清朝的統治與西方列強的衝擊中經歷多次大變動，並在近代開始產生臺灣主體意識思想。"

In [ ]:
result_tw = pipeline.run({
    "prompt_builder": {
        "passage": introduction_tw,
        "schema": json_schema
    }
})

In [ ]:
valid_reply_tw = result_tw["output_validator"]["valid_replies"][0]
valid_json_tw = json.loads(valid_reply_tw)
print(valid_json_tw)